In [1]:
import numpy as np
import requests
import time

from pyspark import SQLContext, SparkContext
from pyspark import SparkConf

from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.linalg import Vectors

from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold

from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors

In [2]:
# Load the dataset and labels
x=np.load('Bx.npy')
y=np.load('By.npy')

In [3]:
conf = SparkConf()
sc = SparkContext(conf = conf)
spark = SQLContext(sc)

In [4]:
# Calculate the rank of each feature
R=[]
for h in range(x.shape[1]):
    kmeans = KMeans(init='k-means++', n_clusters=np.unique(y).shape[0])
    ff=kmeans.fit_predict(x[:,h].reshape(-1,1))
    r=metrics.homogeneity_score(y,ff) #Use the homogeneity score as a rank of the feature
    R.append(r)

In [5]:
#Arrange feature accroding to thier ranks
Rnk=np.argsort(np.array(R))

In [6]:
#Initiate the cross-validation splitter
kfolds=StratifiedKFold(n_splits=5,shuffle=True)

In [7]:
#Per each set of ranks, use cross-validation to calculate accuracy.
smr=[]
et=0
for j in range(Rnk.shape[0]):
    fd=x[:,Rnk[j:]]
    pp=0
    lpa=np.zeros((0,2))
    for train,test in kfolds.split(fd,y):
        dff = map(lambda x: (int(float(x[-1])), Vectors.dense(x[:-1])),np.hstack((fd[train],y[train].reshape(-1,1))))
        TrD = spark.createDataFrame(dff,schema=["label", "features"]).rdd.map(lambda row: LabeledPoint(row.label, MLLibVectors.fromML(row.features)))
        dff = map(lambda x: (int(float(x[-1])), Vectors.dense(x[:-1])),np.hstack((fd[test],y[test].reshape(-1,1))))
        TsD = spark.createDataFrame(dff,schema=["label", "features"]).rdd.map(lambda row: LabeledPoint(row.label, MLLibVectors.fromML(row.features)))
        model = RandomForest.trainClassifier(TrD, numClasses=np.unique(y).shape[0],categoricalFeaturesInfo={},numTrees=100) #The number of classes in the dataset
        predictions = model.predict(TsD.map(lambda x: x.features))
        st = time.time()
        labelsAndPredictions = TsD.map(lambda lp: lp.label).zip(predictions)
        lpa=np.vstack((lpa,labelsAndPredictions.toDF().toPandas()))
        et+=time.time()-st
        acc = labelsAndPredictions.filter(lambda lp: lp[0] == lp[1]).count() / float(TsD.count())
        pp=pp+acc
    pp=pp/kfolds.n_splits
    np.savetxt('F%d.csv'%j,lpa,delimiter=',')
    smr.append([j, pp, et*1000000/x.shape[0]]) #Calculate the time requires to predict a label per each object in uS.

In [8]:
smr

[[43.0, 0.9885, 12.13683833],
 [42.0, 0.9886, 13.25858562],
 [41.0, 0.9885, 11.8600223],
 [40.0, 0.9886, 13.48182144],
 [39.0, 0.9886, 13.03017011],
 [38.0, 0.9885, 13.10078501],
 [37.0, 0.9886, 11.66849725],
 [36.0, 0.9885, 11.65227064],
 [35.0, 0.9885, 11.74869218],
 [34.0, 0.9885, 11.57089904],
 [33.0, 0.9885, 11.48238665],
 [32.0, 0.9886, 11.3607301],
 [31.0, 0.9885, 11.42779459],
 [30.0, 0.9885, 11.2978386],
 [29.0, 0.9885, 11.30046068],
 [28.0, 0.9884, 11.23136591],
 [27.0, 0.9884, 11.10158751],
 [26.0, 0.9884, 11.1218422],
 [25.0, 0.9884, 11.01453423],
 [24.0, 0.9884, 11.29130445],
 [23.0, 0.9884, 11.17720366],
 [22.0, 0.9884, 11.00496097],
 [21.0, 0.9884, 11.22026435],
 [20.0, 0.9884, 11.16865529],
 [19.0, 0.9884, 11.03195968],
 [18.0, 0.9884, 10.75415818],
 [17.0, 0.9884, 10.95911542],
 [16.0, 0.9884, 10.64918846],
 [15.0, 0.9884, 11.16233487],
 [14.0, 0.9884, 11.17720272],
 [13.0, 0.9883, 11.02341948],
 [12.0, 0.9881, 11.0418732],
 [11.0, 0.9816, 10.63487131],
 [10.0, 0.9816,